In [30]:
#Created by: Georgina Hoagland
#Created: Aug 29, 2016
#Last Modified: Sept 27, 2016

#To check: Day counters and calendar conventions
#To add: Units

from QuantLib import *

#Swap rate input

#setup global eval date 
calc_date = Date(27, 9, 2016)
Settings.instance().evaluationDate = calc_date

ois = {   (1,Days): 0.004,
          (1,Weeks): 0.004,
          (2,Weeks): 0.004,
          (3,Weeks): 0.00417,
          (1,Months): 0.004175,
          (2,Months): 0.00429,
          (3,Months): 0.004366,
          (4,Months): 0.004515,
          (5,Months): 0.00472,
          (6,Months): 0.00485,
          (9,Months): 0.00518,
          (12,Months): 0.005445,
          (18,Months): 0.005801,
          (2,Years): 0.006070,
          (3,Years): 0.006560,
          (4,Years): 0.007010,
          (5,Years): 0.007490,
          (7,Years): 0.012512,
          (10,Years): 0.013925,
          (12,Years): 0.014701,
          (15,Years): 0.015529,
          (20,Years): 0.016385,
          (25,Years): 0.016780,
          (30,Years): 0.016989,
          (40,Years): 0.016995,
          (50,Years): 0.017085}

#convert to ql Quote objects
for n,unit in ois.keys():
    ois[(n,unit)] = SimpleQuote(ois[(n,unit)])

swaps = { (1,Years): 0.00900,
          (2,Years): 0.00975,    
          (3,Years): 0.01040,
          (4,Years): 0.01099,
          (5,Years): 0.01160,
          (6,Years): 0.01224,
          (7,Years): 0.01284,
          (8,Years): 0.01343,
          (9,Years): 0.01390,
          (10,Years): 0.01438,
          (15,Years): 0.01610,
          (20,Years): 0.01707,
          (25,Years): 0.01752,
          (30,Years): 0.01778} # 1 Year = 90bps, etc. 


#convert swap quotes from floats to Quote objects
for (n,unit) in swaps.keys():
    swaps[(n,unit)] = SimpleQuote(swaps[(n,unit)])


In [31]:
#Calendar, rules, conventions

calendars = [Argentina, Australia, BespokeCalendar, Brazil, Canada, China, \
             CzechRepublic, Denmark, Finland, Germany, HongKong, Hungary, Iceland, \
             India, Indonesia, Israel, Italy, Japan, JointCalendar, Mexico, NewZealand,\
             Norway, NullCalendar, Poland, Romania, Russia, SaudiArabia, Singapore, \
             Slovakia, SouthAfrica, SouthKorea, Sweden, Switzerland, Taiwan, TARGET, \
             Turkey, Ukraine, UnitedKingdom, UnitedStates, WeekendsOnly]

bus_day_conventions = [Following, ModifiedFollowing, Preceding, ModifiedPreceding, \
                       Unadjusted,HalfMonthModifiedFollowing]

gen_rules = [DateGeneration.Backward, DateGeneration.Forward, DateGeneration.Zero, \
             DateGeneration.ThirdWednesday] 

In [32]:
#Swap info

settlementDays = 2
    
fixedLegNotional = 100 #dollars
fixedLegFrequency = Semiannual
fixedLegTenor = Period(6, Months)
fixedLegAdjustment = ModifiedFollowing
fixedLegDayCounter = Thirty360()

floatingLegSpread = 0.0 #Libor rate - Treasury security of same maturity
floatingLegFrequency = Semiannual
floatingLegTenor = Period(6, Months) #Libor 6M
floatingLegAdjustment = ModifiedFollowing
floatingLegDayCounter = Thirty360()
#When changed to Actual360 to correspond with Libor conventions,\
#NPV for maturities that are traded (i.e. 1, 7, 10, 20, 25Y and not 11, 17, 24Y)\
#moved to the e-12 - e-15 range?


#use index in from lists to choose calendar, convention, etc.
eval_date = calc_date
cal = calendars[-6]() #TARGET

gen_rule = gen_rules[0] # Backward 


end_of_month = True
settlementDate = cal.advance(eval_date, settlementDays, Days)

#Unadjusted calendar for termination purposes
termination_cal = calendars[-18]() #NullCalendar
termination_convention = bus_day_conventions[4] #Unadjusted


In [33]:
#Schedule

#creates dicts with keys of year term and values of termination date/payment schedule 
termination_date = {}
for i in range(1, 31):
    termination_date[i] = termination_cal.advance(settlementDate, Period(i, Years), \
                                                  termination_convention)
fixedSchedule = {}
for i in range(1, 31):
    fixedSchedule[i] = Schedule(settlementDate, termination_date[i], fixedLegTenor, cal, \
                                   fixedLegAdjustment, fixedLegAdjustment, gen_rule, \
                                   end_of_month)
floatingSchedule = {}
for i in range(1, 31):
    floatingSchedule[i] = Schedule(settlementDate, termination_date[i], floatingLegTenor, \
                                   cal, floatingLegAdjustment, floatingLegAdjustment, \
                                   gen_rule, end_of_month)

In [34]:
#Bootstrapping 

#need to change to be able to change currency?

# term structure handles
discountTermStructure = RelinkableYieldTermStructureHandle()
forecastTermStructure = RelinkableYieldTermStructureHandle() 
# RelinkableHandle<YieldTermStructure>

libor6MIndex = USDLibor(Period(6, Months), forecastTermStructure)
print libor6MIndex.dayCounter()
print libor6MIndex.businessDayConvention()

swapHelpers =  [SwapRateHelper(QuoteHandle(swaps[(n,unit)]), Period(n,unit), cal,\
                                     fixedLegFrequency, fixedLegAdjustment, \
                                     fixedLegDayCounter, libor6MIndex)\
                      for (n, unit) in swaps.keys() ]


#PiecewiseYieldCurve does not work in python
#currently using PiecewiseFlatForward - corresponding to \
#PiecewiseYieldCurve<ForwardRate,BackwardFlat> in C++
#check day counter - using fixedLegDayCounter

piecewiseYieldCurve = PiecewiseFlatForward(settlementDays, cal, swapHelpers, \
                                           fixedLegDayCounter)

Actual/360 day counter
1


In [35]:
#Swap additions - linear
#uses averages to assign rates to non-existent swap maturities, i.e. 11Y, 12Y, 23Y
swapsAdditionsLinear = {}
for i in range(11, 31):
    if i % 5 != 0:
        swapsAdditionsLinear [(i, Years)] = \
                        ((swaps[((i//5 + 1) * 5, Years)].value() \
                          - swaps[((i//5)*5, Years)].value()) \
                          *(i % 5)/5) + swaps[((i//5)*5, Years)].value()
for (i, unit) in swapsAdditionsLinear.keys():
    swapsAdditionsLinear[(i, unit)] = SimpleQuote(\
                                            swapsAdditionsLinear[(i, unit)])
            
#both the given swap rates and intermediate values for non-existant maturities        
swapsAll = {}
for (i, unit) in swaps.keys():
    swapsAll[(i, unit)] = swaps[(i, unit)]
for (i, unit) in swapsAdditionsLinear.keys():
    swapsAll[(i, unit)] = swapsAdditionsLinear[(i, unit)]

In [36]:
#Swap Pricing

swapTypes = [VanillaSwap.Payer, VanillaSwap.Receiver]
swapType = swapTypes[0]

discountTermStructure.linkTo(piecewiseYieldCurve)
forecastTermStructure.linkTo(piecewiseYieldCurve)
swapEngine = DiscountingSwapEngine(discountTermStructure)

    
vanillaSwaps = {}
for i in range(1, 31):
    vanillaSwaps[(i, Years)] = VanillaSwap(swapType, fixedLegNotional, fixedSchedule[i],\
                        swapsAll[(i, Years)].value(), fixedLegDayCounter, \
                        floatingSchedule[i], libor6MIndex, floatingLegSpread,\
                        floatingLegDayCounter)
    
    vanillaSwaps[(i, Years)].setPricingEngine(swapEngine)
    
swapsNPV = {}
for i in range(1, 31):
    swapsNPV[(i, Years)] = vanillaSwaps[(i, Years)].NPV()
    print i, swapsNPV[(i, Years)]

1 -0.012213837366
2 -0.0260700711295
3 -0.0417311728725
4 -0.0618323364617
5 -0.0800197059262
6 -0.0997742361885
7 -0.11974985711
8 -0.146762292423
9 -0.16848699167
10 -0.191022740196
11 -0.0839635103881
12 -0.0468755736261
13 -0.0719819839604
14 -0.158218797349
15 -0.307282502135
16 -0.261875570073
17 -0.248039779373
18 -0.267670221506
19 -0.323345732203
20 -0.415406854711
21 -0.40303061652
22 -0.406332847847
23 -0.42488470709
24 -0.460810209346
25 -0.509235529237
26 -0.508351725629
27 -0.515361166563
28 -0.534254109521
29 -0.556835025591
30 -0.588789718216


In [20]:
#Swaption Volatility

#using constant value of 40%
volatilityFlat = 0.40

volatilityStructure = ConstantSwaptionVolatility(settlementDays, cal,\
                                                     fixedLegAdjustment, volatilityFlat, \
                                                     fixedLegDayCounter)
# default: VolatilityType type=ShiftedLognormal, const Real shift=0.0

volatilityStructureHandle = SwaptionVolatilityStructureHandle(volatilityStructure)


In [21]:
#Swaption Inputs

swaptionMaturity = Period(3, Months)
swaptionExpiry = cal.advance(settlementDate, swaptionMaturity, floatingLegAdjustment)
settlementType = Settlement.Physical

#European
swaptionExercise = EuropeanExercise(swaptionExpiry)

In [22]:
#Swaption Object

swaptionPricingEngine = BlackSwaptionEngine(forecastTermStructure, volatilityStructureHandle)

swaptions = {}
for i in range(1, 31):
    swaptions [(i, Years)] = Swaption(vanillaSwaps[(i, Years)], swaptionExercise, \
                                      settlementType)
    
    swaptions[(i, Years)].setPricingEngine(swaptionPricingEngine)

In [23]:
#Swaption Pricing

swaptionsNPV = {}
for maturity in swaptions.keys():
    swaptionsNPV[maturity] = swaptions[maturity].NPV()
    
for i in range(1, 31):
    print i, swaptionsNPV[(i, Years)]

1 0.0651949835941
2 0.140546633181
3 0.223653550294
4 0.312041724298
5 0.409320494605
6 0.515009893238
7 0.626283312415
8 0.741222202908
9 0.857115444478
10 0.978610354348
11 1.15979929086
12 1.30722501485
13 1.42619885294
14 1.51453785665
15 1.57436837183
16 1.71948165468
17 1.84729514914
18 1.95805680411
19 2.05154055592
20 2.12642591686
21 2.2417399356
22 2.34655312035
23 2.44269592484
24 2.5290326522
25 2.60898200958
26 2.70642563441
27 2.79912673263
28 2.88306187511
29 2.96417906981
30 3.04012066554


In [24]:
#Implied Volatility

swaptionHelpers = {}
impliedVols = {}
for i in range(1, 31):
    swaptionHelpers[(i, Years)] = SwaptionHelper(swaptionMaturity, Period(i, Years), \
                                  QuoteHandle(SimpleQuote(volatilityFlat)), \
                                  libor6MIndex, fixedLegTenor, fixedLegDayCounter, \
                                  floatingLegDayCounter, forecastTermStructure)

    swaptionHelpers[(i, Years)].setPricingEngine(swaptionPricingEngine)


#    impliedVols[(i, unit)] = swaptionHelpers[(i, unit)].impliedVolatility(\
#                                                            swaptionsNPV[(i, unit)], \
#                                                            1e-7, 50, 0.0, 10.0)\
                             
#values my be causing runtime errors in the implied vol value                